In [1]:
import requests
from bs4 import BeautifulSoup 
import time
from collections import defaultdict
import re
import csv
from urllib.parse import urljoin
import os
import string
import pandas as pd
from urllib.parse import urljoin
from urllib.robotparser import RobotFileParser
import schedule
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools
from spellchecker import SpellChecker
import json

In [2]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Downloading necessary resources for pre-processing
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sahil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sahil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
res=requests.get('https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/')
print(res)
soup=BeautifulSoup(res.text,'html.parser')
#print(soup.prettify)

<Response [200]>


<h1>##Checking robots file for allowed and disallowed websites

In [4]:
# Fetch the robots.txt file
rp = RobotFileParser()
root_url = 'https://pureportal.coventry.ac.uk'
rp.set_url(urljoin(root_url, "/robots.txt"))
rp.read()
def is_allowed(url):
    return rp.can_fetch("*", url)

<h1>##Crawling CSM Members page

In [5]:
def crawl_persons():
    csm_member_links=set()
    url='https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/persons/'
    profile=requests.get(url)
    if is_allowed(url):
        profile_soup=BeautifulSoup(profile.text,'html.parser')
        tiles=profile_soup.select('.result-container')
        for profiles in tiles:
            profile_links=profiles.find("a", class_="link person").get('href')
            csm_member_links.add(profile_links)
        return csm_member_links

<h1>#authors and their publications

In [16]:
def crawl_author_pub_and_count(pub_author_url_count):
    author_pub_and_count = dict()
    for key in list(pub_author_url_count.keys()):
        authors_portal_page = requests.get('https://pureportal.coventry.ac.uk/en/persons/'+key)
        author_soup = BeautifulSoup(authors_portal_page.text, "html.parser")
        author_name=author_soup.find('h1').text  
        author_pub_and_count[author_name] = pub_author_url_count[key]
            
    #print(author_pub_and_count)    
    with open('article_counts.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Author Name', 'No. of Articles Published'])
        for author, count in author_pub_and_count.items():
            writer.writerow([author, count])
    

<h1>##Crawling Websites

In [17]:
def crawler(root, page_count):
    search_list=[]
    visited_urls = set()
    queue = [root] #this is the queue which initially contains the given seed URL
    count = 0
    csm_member_links=crawl_persons()
    while(queue!=[] and count < page_count):
        url = queue.pop(0)
        if is_allowed(url):
            if url in visited_urls:
                continue
            visited_urls.add(url)
            print("fetching " + url)
            count +=1
            page = requests.get(url)
            soup = BeautifulSoup(page.text, "html.parser")

            tiles=soup.select('.result-container')

            c=1
            for subsections in tiles:

                list_of_records={}
                members = subsections.find_all("a", class_="link person")
                is_member_csm=False
                for member in members:
                    #print(member.get('href'))
                    if member.get('href') in csm_member_links:
                        #print('----------')
                        #print(member.get('href'))
                        is_member_csm=True
                        break
                #print(is_member_csm)
                #count+=1
                    #member_link=members.find('a',class_="link person").get('href')
                    #print(member_link)


                if is_member_csm:
                    p_title=subsections.find('a',{'class':'link'}).get_text()
                    p_link=subsections.find('a',{'class':'link'}).get('href')
                    p_date=subsections.find('span',{'class':'date'}).get_text()
                    p_auth_names=[]
                    p_auth_portals=[]


                    for auth_info in subsections.findAll('a',{'class':'link person'}):
                        p_auth_name=auth_info.get_text()
                        p_auth_portal_link=auth_info.get('href')
                        p_auth_names.append(p_auth_name)
                        p_auth_portals.append(p_auth_portal_link)
                        
                        ##for author and their publishing count
                        p_auth_portal_link=p_auth_portal_link.split('/')[-1]
                        if p_auth_portal_link in pub_author_url_count:
                            pub_author_url_count[p_auth_portal_link] += 1
                        else:
                            pub_author_url_count[p_auth_portal_link] = 1
                        
                    list_of_records['Name of Publication']=p_title
                    list_of_records['Publication Link']=p_link
                    list_of_records['Publication Date']=p_date
                    list_of_records['List of Authors']=p_auth_names
                    list_of_records['Author Pureportal Profile Link']=p_auth_portals
                    search_list.append(list_of_records)
                    c=c+1
        
        else:
            print(f'{url} is not allowed to be crawled')

        #print(author_pub_and_count)        
        for nextpage in soup.findAll('a',attrs={"class": "step"}):
            new_url = nextpage.get('href')
            if(new_url != None and new_url != '/'):
                new_url = urljoin(url, new_url)
                #print("new_url is : ", new_url)  #uncomment the print statement to see the urls
                queue.append(new_url)

        # Sleep for a few seconds to avoid hitting the server too fast
        time.sleep(rp.crawl_delay('*'))

        headers=['Name of Publication','Publication Link','Publication Date','List of Authors','Author Pureportal Profile Link']
        with open('output.csv', mode='w', newline='', encoding='utf-8') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=headers)
            writer.writeheader()
            for record in search_list:
                writer.writerow(record)
    #print(pub_author_url_count)
    crawl_author_pub_and_count(pub_author_url_count)
                

<h1> ##Schedule the crawler

In [9]:

def run_crawler():
    crawl_persons()
    print("Crawler running at", time.strftime("%Y-%m-%d %H:%M:%S"))
    crawler('https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/',10)
    #if it is running on friday
    while True:
        schedule.run_pending()
        time.sleep(60) # Wait for 1 minute

In [18]:
while True:
    x = input('Select the following options\n1. To Run Crawler \n2. Schedule the crawler to run every Friday\n3. Exit\n')
    if x == '1':
        pub_author_url_count=dict()
        
        crawl_persons()
        crawler('https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/', 1)

        print("Crawler running at", time.strftime("%Y-%m-%d %H:%M:%S"))
        print('crawling completed')
        #crawl_author_pub_and_count(auth_dict)
        print()
        print('2 files generated')
        current_dir = os.path.abspath(os.curdir)
        file_path = os.path.join(current_dir, 'output.csv')
        print("File path:", file_path)
        current_dir = os.path.abspath(os.curdir)
        file_path = os.path.join(current_dir, 'article_counts.csv')
        print("check output File path:", file_path)
        break
    elif x == '2':
        schedule.every().friday.at('01:00').do(run_crawler)
        print('crawler scheduled to run every friday at 01:00 AM')
        print()
    
    elif x=='3':
        break
    else:
        print('Choose the correct option')

Select the following options
1. To Run Crawler 
2. Schedule the crawler to run every Friday
3. Exit
 1


fetching https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/
{'Miao Lin Pay': 1, 'Jesper Christensen': 1, 'Fei He': 7, 'Laura Roden': 2, 'Majdi Fanous': 2, 'Jonathan Eden': 2, 'Alireza Daneshkhah': 3, 'Maria Tariq': 1, 'Vasile Palade': 12, 'YingLiang Ma': 1, 'Xiaorui Jiang': 3, 'Sivasharmini Ganeshamoorthy': 1, 'Dominik Klepl': 4, 'Shenal Rajintha Alexander Samarathunge Gunawardena': 1, 'Huma Shah': 1, 'Pengpeng Hu': 1, 'AmirHosein Sadeghimanesh Sadeghi Manesh': 4, 'Jonathan Nixon': 1, 'Elena Gaura': 4, 'Alison Halford': 3, 'Michael Ajao-Olarinoye': 1, 'Ammar Al Bazi': 1, 'Matthew England': 3, 'Ankur Deo': 1, 'Matthew Stephen Tart': 1, 'Anup Pandey': 1, 'James Brusey': 2, 'Abiola Babatunde': 1, 'Brandi Jo Jess': 1, 'Beate Grawemeyer': 1, 'John Halloran': 1, 'Rashid Barket': 1, 'James Donnelly': 1, 'Omid Chatrabgoun': 1}
Crawler running at 2023-03-30 02:51:24
crawling completed

2 files generated
File path: C

File path: C:\Users\Sahil\Documents\Information retrieval\output.csv


In [11]:
output=pd.read_csv("output.csv") 
output.head(2)

,Name of Publication,Publication Link,Publication Date,List of Authors,Author Pureportal Profile Link
0,An Extended Plant Circadian Clock Model for Ch...,https://pureportal.coventry.ac.uk/en/publicati...,9 Jan 2023,"['Pay, M. L.', 'Christensen, J.', 'He, F.', 'R...",['https://pureportal.coventry.ac.uk/en/persons...
1,Challenges and prospects of climate change imp...,https://pureportal.coventry.ac.uk/en/publicati...,25 Feb 2023,"['Fanous, M.', 'Eden, J.', 'Daneshkhah, A.']",['https://pureportal.coventry.ac.uk/en/persons...


In [12]:
##preprocessing
#This code performs the following preprocessing steps:

#Removes Unicode characters from the text.
#Removes mentions (words starting with '@').
#Converts the text into lowercase.
#Removes punctuation marks.
#Removes stop words (common words like "the", "a", "an", etc.).
#Joins the remaining words back into a string.



processed_pub_names = []
def preprocess_text(pub_name):
    # Removing non-ASCII characters
    pub_name = pub_name.encode('ascii', 'ignore').decode()
    # Removing mentions (starting with '@')
    pub_name = re.sub(r'@\w+', '', pub_name)
    # Converting to lowercase
    pub_name = pub_name.lower()
    # Removing punctuation marks
    pub_name = pub_name.translate(str.maketrans('', '', string.punctuation))
    # Removing stop words
    stop_words = set(stopwords.words('english'))
    pub_name = ' '.join(word for word in pub_name.split() if word not in stop_words)
    # Stemming words
    stemmer = PorterStemmer()
    pub_name = ' '.join(stemmer.stem(word) for word in pub_name.split())
    return pub_name

# apply the pre-processing function to each publication name and store the results in the processed_pub_names list
for pub_name in output['Name of Publication']:
    processed_pub_name = preprocess_text(pub_name)
    processed_pub_names.append(processed_pub_name)
    
# create an empty dictionary to hold the inverted index
inverted_index = {}

# iterate through each publication name and tokenize it
for i, pub_name in enumerate(processed_pub_names):
    tokens = pub_name.split()
    # iterate through each token and update the inverted index
    for token in tokens:
        if token in inverted_index:
            inverted_index[token].append(i)
        else:
            inverted_index[token] = [i]

# print top 5 entries in the inverted index
top_five = dict(itertools.islice(inverted_index.items(), 5))

print(top_five)



{'extend': [0], 'plant': [0, 30], 'circadian': [0], 'clock': [0], 'model': [0, 1, 6, 30, 31, 38, 71, 78, 80, 97]}


In [13]:
tfid = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None, binary=False)
tfid_matrix = tfid.fit_transform(processed_pub_names)
terms = tfid.get_feature_names()
dense_matrix = tfid_matrix.toarray()
dense_list = dense_matrix.tolist()

# Create a dataframe with the tf-idf values
tf_idf_df = pd.DataFrame(dense_list, columns=terms)

C:\Users\Sahil\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [38]:
from autocorrect import Speller

spell = Speller(lang='en')

inputquery = input('Enter Publication Name: ')
enteredquery=inputquery
corrected_query=[]
for word in inputquery.split(' '):
    corrected_word = spell(word)
    corrected_query.append(corrected_word)
inputquery=' '.join(corrected_query)

clean_inputquery = preprocess_text(inputquery).split()
    
print(clean_inputquery)


Enter Publication Name:  machine larninng


['machin', 'learn']


In [39]:
# Calculate match scores
match_scores = [sum(tf_idf_df.iloc[i][j] for j in clean_inputquery if j in tf_idf_df.columns) for i in range(len(processed_pub_names))]

# Check if there are any matches
if all(score == 0 for score in match_scores):
    print("No related research paper found.")
else:
    # Sort results by match score and select top 6
    top_results = sorted(enumerate(match_scores), key=lambda x: x[1], reverse=True)[:6]
    
    # Print results
    print('Entered Query:',enteredquery)
    print('showing results for',' '.join(clean_inputquery))
    print()
    pd.set_option('display.max_colwidth', None)
    for i, score in top_results:
        row = output.iloc[i]
        publication_name = row['Name of Publication'].strip()
        publication_link = row['Publication Link'].strip()
        publication_date = row['Publication Date'].strip()
        authors = row['List of Authors'].strip()
        authors_portal_links = row['Author Pureportal Profile Link'].strip()
        print('Publication Name:', publication_name)
        print('Publication Link:', publication_link)
        print('Publication Date:', publication_date)
        print('Authors:', authors)
        print('Author Portal Links:', authors_portal_links)
        print("***********************************************")

Entered Query: machine larninng
showing results for machin learn

Publication Name: Machine Learning for Computer Algebra
Publication Link: https://pureportal.coventry.ac.uk/en/publications/machine-learning-for-computer-algebra
Publication Date: 2022
Authors: ['Barket, R.', 'del Río, T.', 'England, M.']
Author Portal Links: ['https://pureportal.coventry.ac.uk/en/persons/rashid-barket', 'https://pureportal.coventry.ac.uk/en/persons/tereso-del-r%C3%ADo-almajano', 'https://pureportal.coventry.ac.uk/en/persons/matthew-england']
***********************************************
Publication Name: SC-Square: Future Progress with Machine Learning
Publication Link: https://pureportal.coventry.ac.uk/en/publications/sc-square-future-progress-with-machine-learning
Publication Date: 14 Nov 2022
Authors: ['England, M.']
Author Portal Links: ['https://pureportal.coventry.ac.uk/en/persons/matthew-england']
***********************************************
Publication Name: Using Machine Learning in SC2
Pu